# STA 141B Data & Web Technologies for Data Analysis


### Lecture 8, 2/5/24, APIs


### Last week's topics

- APIs

### Today's topics

- Undocumented APIs

### Ressources
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

### Recap: HTTP

A response to an HTTP request always includes a status code that summarizes whether the request was successful. Wikipedia has a full [list of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). Generally,

* 200-299: Your request succeeded.
* 300-399: You need to take further action to complete the request.
* 400-499: Your request wasn't valid (you made a mistake). You've probably seen 404 before!
* 500-599: Your request failed (the server made a mistake).

In [1]:
import requests

In [2]:
response = requests.get("https://itunes.apple.com/search", params = {
        "term": "beyonce", 
        "media": "music",
        "entity": "album",
        "attribute": "artistTerm", 
        "country": "US", 
        "limit": "1"
    })

In [3]:
response.url

'https://itunes.apple.com/search?term=beyonce&media=music&entity=album&attribute=artistTerm&country=US&limit=1'

### Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

 - [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

You can identify these websites by looking at requests in your browser's developer tools. For Firefox and Chrome these can be accessed (Windows: <kbd>Ctrl</kbd> + <kbd>i</kbd>; MacOS: <kbd>&#8984;</kbd> + <kbd>&#8997;</kbd> + <kbd>i</kbd>).

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

Let's reverse engineer the Yolo County Health Inspections web API so that we can get data about local restaurants.

In [4]:
import numpy as np
import pandas as pd
import requests
import requests_cache
requests_cache.install_cache("lecture8")

In [5]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [11]:
result = requests.post(url, params = {
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4'
}, data = {
    'FacilityName': "raising cane's"
})
result.raise_for_status()

Check the [docs](https://requests.readthedocs.io/en/latest/api/?highlight=post#requests.post) for `requests`!

In [7]:
result.url

'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities?PressAgentOid=c08cb189-894c-4c8c-b595-a5ef010226b4'

In [8]:
result.json()

[{'FacilityId': 'FA0021137',
  'FacilityName': "RAISING CANE'S #552",
  'Address': '207 E ST ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': 'e95f81ff-6ee7-4194-8de1-b0df010d369a'}]

Lets investigate this further. The second request uses the `FacilityID` as parameter. 

In [14]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/programs'
result = requests.get(url, params = {
    'FacilityId': 'FA0021137', 
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4'
})
result.raise_for_status()
result.json()

[{'ProgramId': 'PR0067650',
  'ProgramIdentifier': None,
  'ProgramCategory': 'FOOD INSPECTION',
  'LastScore': 100.0,
  'attachmentId': 'e95f81ff-6ee7-4194-8de1-b0df010d369a'}]

In [15]:
result.url

'https://yoloeco.envisionconnect.com/api/pressAgentClient/programs?FacilityId=FA0021137&PressAgentOid=c08cb189-894c-4c8c-b595-a5ef010226b4'

We are interested in the inspections text, for which we have to provide the `ProgramID` parameter. 

In [16]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections'

In [17]:
result = requests.get(url, params = {
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
    'ProgramId': 'PR0067650'
})
result.raise_for_status()

In [18]:
results = result.json()
results

[{'activity_date': '2023-12-21T00:00:00',
  'service': 'FOLLOWUP INSPECTION',
  'Oid': 'c513cf46-6df0-43c4-be93-b0df010c8dbf',
  'FacilityOid': 'ba04187b-5af0-4e75-a683-ac9300b481c1',
  'score': 100.0,
  'attachmentId': 'e95f81ff-6ee7-4194-8de1-b0df010d369a',
  'color': 'Green',
  'violations': []},
 {'activity_date': '2023-12-20T00:00:00',
  'service': 'ROUTINE-INSPECTION',
  'Oid': 'f1ee1915-4952-4df2-9657-b0de0109f3bd',
  'FacilityOid': 'ba04187b-5af0-4e75-a683-ac9300b481c1',
  'score': 100.0,
  'attachmentId': '17f76867-01ae-44c8-88d7-b0de010d7987',
  'color': 'Green',
  'violations': [{'violation_description': 'K007 - Proper hot and cold holding temperatures',
    'v_memo': 'California Retail Food Code §113996. Observed coleslaw holding at 54 degree F in a preparation unit. Per manager, coleslaws were prepared and placed in the unit about 2 hours prior the inspection. Manager also stated that this unit was not operable. All potentially hazardous foods shall be maintained at or bel

In [19]:
results_df = pd.DataFrame(results)
results_df

,activity_date,service,Oid,FacilityOid,score,attachmentId,color,violations
0,2023-12-21T00:00:00,FOLLOWUP INSPECTION,c513cf46-6df0-43c4-be93-b0df010c8dbf,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,e95f81ff-6ee7-4194-8de1-b0df010d369a,Green,[]
1,2023-12-20T00:00:00,ROUTINE-INSPECTION,f1ee1915-4952-4df2-9657-b0de0109f3bd,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,17f76867-01ae-44c8-88d7-b0de010d7987,Green,[{'violation_description': 'K007 - Proper hot ...
2,2023-05-22T00:00:00,ROUTINE-INSPECTION,8eac3486-7a8a-41b5-9620-b00a0156c4dd,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,c4c05816-d8af-4fae-bbee-b00b00fe5e54,Green,[]
3,2022-08-30T00:00:00,FOLLOWUP INSPECTION,e6ea660a-08a3-4f95-b5fd-af0101006c45,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,78b377d4-81cd-45fa-8a13-af020097dbb1,Green,[]
4,2022-08-25T00:00:00,ROUTINE-INSPECTION,08077d35-cbc0-4a16-9914-aefc00d37ac4,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,8660b6d1-18a1-4774-9c0c-aefc01117cb8,Green,[{'violation_description': 'K007 - Proper hot ...
5,2021-12-10T00:00:00,ROUTINE-INSPECTION,d8041dec-f7b6-48f2-8ff0-adfa00b92bad,ba04187b-5af0-4e75-a683-ac9300b481c1,100.0,8b7ad724-e57b-48ed-96b9-adfa00fe7100,Green,[{'violation_description': 'K035 - Equipment/U...


In [27]:
results_df['violations'][5]

[{'violation_description': 'K035 - Equipment/Utensils - approved installed clean good repair, capacity',
  'v_memo': 'California Retail Food Code §114175. The walk in condenser has heavy ice build up. Equipment and utensils shall be kept clean, fully operative, and in good repair. Equipment must be repaired and in use or removed from the facility. Correct within 2 weeks.\r\n',
  'violation_text': 'All utensils and equipment shall be fully operative and in good repair. (114175). All utensils and equipment shall be approved, installed properly, and meet applicable standards. (114130, 114130.1, 114130.2, 114130.3, 114130.4, 114130.5, 114132, 114133, 114137, 114139, 114153, 114155, 114163, 114165, 114167, 114169, 114177, 114180, 114182)',
  'DailyOid': 'd8041dec-f7b6-48f2-8ff0-adfa00b92bad',
  'violation_degree': None,
  'Oid': '6030ea1b-8b82-4530-a76d-adfa00b9519a'},
 {'violation_description': 'K038 - Adequate ventilation and lighting designated area, use',
  'v_memo': 'California Retail 

In [21]:
results_df['violations'][1][0]['v_memo']

'California Retail Food Code §113996. Observed coleslaw holding at 54 degree F in a preparation unit. Per manager, coleslaws were prepared and placed in the unit about 2 hours prior the inspection. Manager also stated that this unit was not operable. All potentially hazardous foods shall be maintained at or below 41 degrees F for cold holding or at or above 135 degrees F for hot holding at all times. See temperature log for temperatures taken during the inspection.  Observed manager instructing employee to move all coleslaw to the walk in. This preparation unit shall be repaired/replaced and reinspected by this office prior storing any potential hazardous foods (PHFs).\r\n'

In [22]:
len(results_df['violations'][1])

1

In [28]:
violations = [
    results_df['violations'][5][i]['violation_description'] for i in range(len(results_df['violations'][5]))
]
violations

['K035 - Equipment/Utensils - approved installed clean good repair, capacity',
 'K038 - Adequate ventilation and lighting designated area, use']

In [29]:
{'RAISING CANE\'S #552': violations}

{"RAISING CANE'S #552": ['K035 - Equipment/Utensils - approved installed clean good repair, capacity',
  'K038 - Adequate ventilation and lighting designated area, use']}

How can we generalize this procedure? 

In [30]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [ ]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"
}, 
                     data = {
    "FacilityName": "Ali Baba", 
})
result.raise_for_status()

In [ ]:
result.json()

In [31]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
              data = {
    "FacilityName": "a", 
})
result.json()

[{'FacilityId': 'FA0001345',
  'FacilityName': 'A&B LIQUOR',
  'Address': '2328 W CAPITOL AVE ',
  'CityStateZip': 'WEST SACRAMENTO CA 95691 ',
  'LastScore': 100.0,
  'attachmentId': 'c3afdd86-26f1-4cd1-859a-b0f300b80b46'},
 {'FacilityId': 'FA0022329',
  'FacilityName': 'ACE SUSHI @SAVE MART 604',
  'Address': '1900 ANDERSON RD ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '0ee7d4cf-1007-4f26-a6e3-b0f901141236'},
 {'FacilityId': 'FA0019474',
  'FacilityName': 'ACOUSTIC EVENTS',
  'Address': '4467 D ST ',
  'CityStateZip': 'SACRAMENTO CA 95819 ',
  'LastScore': 100.0,
  'attachmentId': '6ad8b394-103b-4f88-8e1c-ae2b01015500'},
 {'FacilityId': 'FA0014014',
  'FacilityName': 'AFC SUSHI / HOT WOK @ BEL AIR #526',
  'Address': '1885 E GIBSON RD ',
  'CityStateZip': 'WOODLAND CA 95776 ',
  'LastScore': 100.0,
  'attachmentId': 'b935b8cb-815a-4c30-8a00-b05701044f13'},
 {'FacilityId': 'FA0014013',
  'FacilityName': "AFC SUSHI / HOT WOK @ RALEY'S #206",
  'Addr

In [32]:
pd.DataFrame(result.json())

,FacilityId,FacilityName,Address,CityStateZip,LastScore,attachmentId
0,FA0001345,A&B LIQUOR,2328 W CAPITOL AVE,WEST SACRAMENTO CA 95691,100.0,c3afdd86-26f1-4cd1-859a-b0f300b80b46
1,FA0022329,ACE SUSHI @SAVE MART 604,1900 ANDERSON RD,DAVIS CA 95616,100.0,0ee7d4cf-1007-4f26-a6e3-b0f901141236
2,FA0019474,ACOUSTIC EVENTS,4467 D ST,SACRAMENTO CA 95819,100.0,6ad8b394-103b-4f88-8e1c-ae2b01015500
3,FA0014014,AFC SUSHI / HOT WOK @ BEL AIR #526,1885 E GIBSON RD,WOODLAND CA 95776,100.0,b935b8cb-815a-4c30-8a00-b05701044f13
4,FA0014013,AFC SUSHI / HOT WOK @ RALEY'S #206,367 W MAIN ST,WOODLAND CA 95695,100.0,4f049df9-a074-4cc2-a699-b0ed010184ef
5,FA0014015,AFC SUSHI / HOT WOK @ RALEY'S #448,1601 W CAPITOL AVE,WEST SACRAMENTO CA 95691,100.0,54ab2356-9d16-40fa-a278-b0900101971d
6,FA0010191,AFC SUSHI @ SAFEWAY #1205,1451 W COVELL BLVD,DAVIS CA 95616,100.0,27932423-7d36-4111-b7d3-b10000a2a956
7,FA0010190,AFC SUSHI @ SAFEWAY #1561,2121 COWELL BLVD,DAVIS CA 95616,100.0,2413349b-44ec-42a2-86a7-b10e00916b1e
8,FA0018619,AFC TIK TOK WOK @ SAFEWAY #1205,1451 W COVELL BLVD,DAVIS CA 95616,100.0,79862b84-0d2a-425d-ac88-b10000a0018b
9,FA0012949,AFRIDI FOOD COMPANY,1250 CHURCHILL DOWNS AVE D STE,WOODLAND CA 95776,100.0,b55fd798-09fb-4bbb-9c4d-aff1010b75ae


Lets write a pipeline. 

In [39]:
def fetch_violations(ProgramId):
    result = requests.get(
        'https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': ProgramId
    })
    result.raise_for_status()
    results = result.json()
    results_df = pd.DataFrame(results)
    violations = [
        results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))
    ]
    return(violations)

In [41]:
fetch_violations('PR0067650') # for in-n-out

[]

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0024103'
    })

In [ ]:
result.text

In [37]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    })
    result.raise_for_status()
    ProgramId = result.json()[0]['ProgramId']
    return(ProgramId)

In [38]:
fetch_ProgramId('FA0016070')

'PR0053504'

In [33]:
def fetch_FacilityID(letter):
    result = requests.post(
        'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities', 
                           params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
                           data = {
    "FacilityName": letter, 
    })
    facility_table = pd.DataFrame(result.json())[['FacilityId', 'FacilityName']]
    return(facility_table)

In [36]:
fetch_FacilityID('a')

,FacilityId,FacilityName
0,FA0001345,A&B LIQUOR
1,FA0022329,ACE SUSHI @SAVE MART 604
2,FA0019474,ACOUSTIC EVENTS
3,FA0014014,AFC SUSHI / HOT WOK @ BEL AIR #526
4,FA0014013,AFC SUSHI / HOT WOK @ RALEY'S #206
5,FA0014015,AFC SUSHI / HOT WOK @ RALEY'S #448
6,FA0010191,AFC SUSHI @ SAFEWAY #1205
7,FA0010190,AFC SUSHI @ SAFEWAY #1561
8,FA0018619,AFC TIK TOK WOK @ SAFEWAY #1205
9,FA0012949,AFRIDI FOOD COMPANY


In [42]:
import time

In [43]:
[letter for letter in map(chr, range(97, 99))]

['a', 'b']

In [44]:
x = {}
type(x)

dict

In [45]:
def get_violations(): 
    violations = {}
    for letter in map(chr, range(97, 99)): # map(chr, range(97, 123)) takes too long
        time.sleep(0.05) # sleep until making a request for each letter
        facility_table = fetch_FacilityID(letter)
        for index in range(facility_table.shape[0]): # for all facilities returned for this letter
            FacilityId, FacilityName = facility_table.iloc[index]
            time.sleep(0.1) # sleep again for each individual request
            ProgramId = fetch_ProgramId(FacilityId)
            print(FacilityName)
            violations[FacilityName] = fetch_violations(ProgramId)
    return(violations)

In [46]:
violations = get_violations()

A&B LIQUOR


KeyError: 'violations'

In [47]:
fetch_FacilityID('A&B LIQUOR')

,FacilityId,FacilityName
0,FA0001345,A&B LIQUOR


In [49]:
fetch_ProgramId('FA0001345')

'PR0000623'

In [50]:
ProgramId = fetch_ProgramId('FA0001345')            
ProgramId

'PR0000623'

In [51]:
fetch_violations('PR0000623')

KeyError: 'violations'

In [52]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0000623'
})
result.raise_for_status()

In [53]:
results = result.json()
results

[]

Lets check this in the browser! 

In [54]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': 'FA0001345'
    }).json()
[result[i]['ProgramId'] for i in range(len(result))]

['PR0000623', 'PR0069422']

In [55]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    }).json()
    ProgramId = [result[i]['ProgramId'] for i in range(len(result))]
    return(ProgramId)

In [56]:
fetch_ProgramId('FA0001345')

['PR0000623', 'PR0069422']

In [57]:
def fetch_violations(ProgramId_list):
    violations = []
    for ProgramId in ProgramId_list: 
        result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
            'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
            'ProgramId': ProgramId
        }).json()
        results_df = pd.DataFrame(result)
        if not results_df.empty: # only append violations if there are any
            violations.extend(
                [results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))]
            )
    return(violations)

In [58]:
fetch_violations(['PR0000623', 'PR0069422'])

['K006 - Adequate handwashing facilities supplied & accessible',
 'K007 - Proper hot and cold holding temperatures',
 'K021 - Hot and cold water available']

In [59]:
violations = get_violations()

A&B LIQUOR
ACE SUSHI @SAVE MART 604
ACOUSTIC EVENTS
AFC SUSHI / HOT WOK @ BEL AIR #526
AFC SUSHI / HOT WOK @ RALEY'S #206
AFC SUSHI / HOT WOK @ RALEY'S #448
AFC SUSHI @ SAFEWAY #1205
AFC SUSHI @ SAFEWAY #1561
AFC TIK TOK WOK @ SAFEWAY #1205
AFRIDI FOOD COMPANY
AFTER HOURS BOBA & TEA
AGGIE LIQUOR
AGTECH INNOVATION ALLIANCE
AISLE 1 #2576
AJ HUNDAL MART
AKIRA COFFEE & TEA
ALI BABA RESTAURANT
ALL SEASONS ALL REASONS CATERING
ALOHA POKEE & RAMEN
ALYCE NORMAN SCHOOL
AM/PM MINI MARKET #5731- FOOD
AMY LOVES MUSTARD
ANAR PERSIAN KITCHEN
ANDERSON FAMILY CATERING BBQ
ANDERSON GAS & MINI MART - FOOD
ANDERSON ROAD SHELL - FOOD
ANDY'S ARCO - FOOD
ANTOJITOS JAIMITO #4TB1720
APNA BAZAAR
APPLEBEE'S - WOODLAND
ARCO AMPM GAS STATION
ARIANA FOOD MARKET
ARMADILLO MUSIC INC.
ASOCIACION LOS CAPORALES - ARENA CONCESSION
ATLAS CRAFT COFFEE
AUNTIE YASY'S GLUTEN-FREE GOODIES & MEAL DELIVERY LLC
AUTHENTIC INDIA
AVOCADO TOAST
AY! JALISCO TAQUERIA #1
BABES.BUBBLES.BOARDS
BABY O'S DONUTS
BAKLAVA AND COFFEE
BALLAST P

In [ ]:
violations

#### Safeway

Check the [docs](https://requests.readthedocs.io/en/latest/api/?requests.get)!

In [63]:
url = 'https://www.safeway.com/abs/pub/xapi/pgmsearch/v1/search/products'
params = {
    'request-id': 9401706033563384632,
    'q': 'eggs',
    'rows': 30,
    'start': 0,
    'search-type': 'keyword',
    'storeid': 3132,
    'featured': 'true',
    'url': 'https://www.safeway.com',
    'pageurl': 'https://www.safeway.com', 
    'search-uid': 'uid%3D3640904575678%3Av%3D12.0%3Ats%3D1674581210532%3Ahc%3D3', 
    'pagename': 'search',
    'dvid': 'web-4.1search',
}
header = {
    'Ocp-Apim-Subscription-Key': '5e790236c84e46338f4290aa1050cdd4', 
}

In [64]:
results = requests.get(url, params=params)
results.raise_for_status()

In [66]:
params = 'request-id=3671707253905370318&url=https://www.safeway.com&pageurl=https://www.safeway.com&pagename=search&rows=30&start=0&search-type=keyword&storeid=3132&featured=true&search-uid=&q=eggs&sort=&featuredsessionid=&screenwidth=600&dvid=web-4.1search&channel=instore&wineshopstoreid=5799&wineshopwidgetid=nlvkox9e&timezone=America/Los_Angeles&zipcode=94611&visitorId=d42daa35-78ae-48dc-9df7-3c163dd79bc2&pgm=intg-search,wineshop&banner=safeway&variant=ACIP134410_a'

In [68]:
url + '?' + params

'https://www.safeway.com/abs/pub/xapi/pgmsearch/v1/search/products?request-id=3671707253905370318&url=https://www.safeway.com&pageurl=https://www.safeway.com&pagename=search&rows=30&start=0&search-type=keyword&storeid=3132&featured=true&search-uid=&q=eggs&sort=&featuredsessionid=&screenwidth=600&dvid=web-4.1search&channel=instore&wineshopstoreid=5799&wineshopwidgetid=nlvkox9e&timezone=America/Los_Angeles&zipcode=94611&visitorId=d42daa35-78ae-48dc-9df7-3c163dd79bc2&pgm=intg-search,wineshop&banner=safeway&variant=ACIP134410_a'

In [69]:
results = requests.get(url + '?' + params, headers=header)
results.json()

{'appMsg': '[PS: Success.]',
 'pgmList': [{'response': {'numFound': 0,
    'start': 0,
    'miscInfo': {'attributionToken': 'rQHwrAoMCP_Giq4GEOnGrtoCEAEaJDY1YzZkZDI2LTAwMDAtMjA4NS1iYzgyLTE0YzE0ZWYxZmJmNCokZDQyZGFhMzUtNzhhZS00OGRjLTlkZjctM2MxNjNkZDc5YmMyMizHy_MX9pmEIqOAlyK4maEijr6dFaaL7xfC8J4V1LKdFYz3pyKpwP0sifenIjobd2ViLXdpbmVzaG9wLXNlcnZpbmctY29uZmlnSAFYAWgB',
     'query': 'eggs',
     'filter': '(inventory(5799, price) >0) AND (inventory(5799,attributes.ship_to_store) = 1) AND (inventory(5799, attributes.shipping): ANY("1", "2") AND  ((attributes.wineEligible: ANY("WE","ALL")))  AND attributes.is_market_place_product : ANY("false"))'},
    'shippingInfo': {'arriveByTS': '2024-02-08T20:00:00Z',
     'displayArriveByTS': '2024-02-08T12:00:00',
     'shippingType': 'REGULAR',
     'minArriveByTS': '2024-02-07T20:00:00Z',
     'displayMinArriveByTS': '2024-02-07T12:00:00'},
    'isExactMatch': False},
   'appCode': 'GR204 PS: 200',
   'appMsg': 'GR : Google Retail returned empty respons

In [ ]:
results.raise_for_status

In [ ]:
'''[
    {asdf}
    {aasd}
]'''

In [ ]:
'''[
    {asdf}
]'''

'''[
    {asdf}
]'''

In [ ]:
from json import loads

In [74]:
loads('[{"a":1}]')

[{'a': 1}]

### Summary 

- Check the query type, header and params using the developer tools 
- Often, multiple API queries are made to display one result 